In [ ]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import BernoulliNB
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFE
import itertools

from xgboost import XGBClassifier
from tabulate import tabulate

In [ ]:
#import dataset
df=pd.read_csv('/content/Train_data.csv.zip')
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,0,tcp,exec,RSTO,0,0,0,0,0,0,...,7,0.03,0.06,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
25188,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,39,1.00,0.00,1.00,0.18,0.00,0.00,0.00,0.00,anomaly
25189,0,tcp,private,REJ,0,0,0,0,0,0,...,13,0.05,0.07,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
25190,0,tcp,nnsp,S0,0,0,0,0,0,0,...,20,0.08,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


In [ ]:
#Feature selection
df=df.drop(['land','wrong_fragment','urgent','hot','num_failed_logins','num_compromised','root_shell','su_attempted','num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login'
],axis=1)

In [ ]:
#Visualsing the Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  object 
 2   service                      25192 non-null  object 
 3   flag                         25192 non-null  object 
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   logged_in                    25192 non-null  int64  
 7   count                        25192 non-null  int64  
 8   srv_count                    25192 non-null  int64  
 9   serror_rate                  25192 non-null  float64
 10  srv_serror_rate              25192 non-null  float64
 11  rerror_rate                  25192 non-null  float64
 12  srv_rerror_rate              25192 non-null  float64
 13  same_srv_rate   

In [ ]:
value_counts = df['service'].value_counts()
value_counts

http         8003
private      4351
domain_u     1820
smtp         1449
ftp_data     1396
             ... 
urh_i           4
red_i           3
pm_dump         3
tim_i           2
http_8001       1
Name: service, Length: 66, dtype: int64

In [ ]:
df.describe()

,duration,src_bytes,dst_bytes,logged_in,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,25192.000000,2.519200e+04,2.519200e+04,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,...,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000
mean,305.054104,2.433063e+04,3.491847e+03,0.394768,84.591180,27.698754,0.286338,0.283762,0.118630,0.120260,...,182.532074,115.063036,0.519791,0.082539,0.147453,0.031844,0.285800,0.279846,0.117800,0.118769
std,2686.555640,2.410805e+06,8.883072e+04,0.488811,114.673451,72.468242,0.447312,0.447599,0.318745,0.322335,...,98.993895,110.646850,0.448944,0.187191,0.308367,0.110575,0.445316,0.446075,0.305869,0.317333
min,0.000000,0.000000e+00,0.000000e+00,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.000000e+00,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,84.000000,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,4.400000e+01,0.000000e+00,0.000000,14.000000,8.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,61.000000,0.510000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.790000e+02,5.302500e+02,1.000000,144.000000,18.000000,1.000000,1.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000
max,42862.000000,3.817091e+08,5.151385e+06,1.000000,511.000000,511.000000,1.000000,1.000000,1.000000,1.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.describe(include='object')

,protocol_type,service,flag,class
count,25192,25192,25192,25192
unique,3,66,11,2
top,tcp,http,SF,normal
freq,20526,8003,14973,13449


In [ ]:
df.isnull().sum()

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
logged_in                      0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_host_rate             0
dst_host_count                 0
dst_host_srv_count             0
dst_host_same_srv_rate         0
dst_host_diff_srv_rate         0
dst_host_same_src_port_rate    0
dst_host_srv_diff_host_rate    0
dst_host_serror_rate           0
dst_host_srv_serror_rate       0
dst_host_rerror_rate           0
dst_host_srv_rerror_rate       0
class                          0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
#Finding Z-score for the dataset
import pandas as pd
from scipy.stats import zscore
numeric_cols = df.select_dtypes(include=['float64', 'int64'])
z_scores = numeric_cols.apply(zscore)
df_with_zscores = pd.concat([df.drop(columns=numeric_cols.columns), z_scores.add_suffix('_zscore')], axis=1)
df_with_zscores

,protocol_type,service,flag,class,duration_zscore,src_bytes_zscore,dst_bytes_zscore,logged_in_zscore,count_zscore,srv_count_zscore,...,dst_host_count_zscore,dst_host_srv_count_zscore,dst_host_same_srv_rate_zscore,dst_host_diff_srv_rate_zscore,dst_host_same_src_port_rate_zscore,dst_host_srv_diff_host_rate_zscore,dst_host_serror_rate_zscore,dst_host_srv_serror_rate_zscore,dst_host_rerror_rate_zscore,dst_host_srv_rerror_rate_zscore
0,tcp,ftp_data,SF,normal,-0.113551,-0.009889,-0.039310,-0.807626,-0.720244,-0.354628,...,-0.328634,-0.813985,-0.779157,-0.280673,0.073120,-0.287993,-0.641804,-0.627365,-0.221668,-0.374281
1,udp,other,SF,normal,-0.113551,-0.010032,-0.039310,-0.807626,-0.624317,-0.368427,...,0.732059,-1.030895,-1.157831,2.764403,2.375620,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281
2,tcp,private,S0,anomaly,-0.113551,-0.010093,-0.039310,-0.807626,0.334947,-0.299430,...,0.732059,-0.804947,-0.935081,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281
3,tcp,http,SF,normal,-0.113551,-0.009996,0.052473,1.238197,-0.694082,-0.313230,...,-1.540854,1.264742,1.069663,-0.440940,-0.380894,0.073759,-0.574435,-0.604947,-0.385140,-0.342768
4,tcp,http,SF,normal,-0.113551,-0.010010,-0.034582,1.238197,-0.476067,0.059355,...,0.732059,1.264742,1.069663,-0.440940,-0.478183,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,tcp,exec,RSTO,anomaly,-0.113551,-0.010093,-0.039310,-0.807626,0.134374,-0.285631,...,0.732059,-0.976667,-1.091006,-0.120406,-0.478183,-0.287993,-0.641804,-0.627365,2.884296,2.777041
25188,tcp,ftp_data,SF,anomaly,-0.113551,-0.009954,-0.039310,1.238197,-0.728964,-0.368427,...,-1.833807,-0.687453,1.069663,-0.440940,2.764775,1.339894,-0.641804,-0.627365,-0.385140,-0.374281
25189,tcp,private,REJ,anomaly,-0.113551,-0.010093,-0.039310,-0.807626,0.177977,-0.285631,...,0.732059,-0.922440,-1.046456,-0.066984,-0.478183,-0.287993,-0.641804,-0.627365,2.884296,2.777041
25190,tcp,nnsp,S0,anomaly,-0.113551,-0.010093,-0.039310,-0.807626,0.387271,-0.133837,...,0.732059,-0.859174,-0.979631,-0.120406,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281


In [ ]:
numeric_cols = df_with_zscores.select_dtypes(include=['float64', 'int64'])
selected_numeric_columns = numeric_cols.columns[((numeric_cols <= 3) & (numeric_cols >= -3)).all()].tolist()
non_numeric_cols = df_with_zscores.select_dtypes(exclude=['float64', 'int64']).columns.tolist()
data = pd.concat([df_with_zscores[non_numeric_cols],df_with_zscores[selected_numeric_columns]], axis=1)
data

,protocol_type,service,flag,class,logged_in_zscore,serror_rate_zscore,srv_serror_rate_zscore,rerror_rate_zscore,srv_rerror_rate_zscore,same_srv_rate_zscore,dst_host_count_zscore,dst_host_srv_count_zscore,dst_host_same_srv_rate_zscore,dst_host_same_src_port_rate_zscore,dst_host_serror_rate_zscore,dst_host_srv_serror_rate_zscore,dst_host_rerror_rate_zscore,dst_host_srv_rerror_rate_zscore
0,tcp,ftp_data,SF,normal,-0.807626,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.328634,-0.813985,-0.779157,0.073120,-0.641804,-0.627365,-0.221668,-0.374281
1,udp,other,SF,normal,-0.807626,-0.640142,-0.633978,-0.372186,-0.373098,-1.320567,0.732059,-1.030895,-1.157831,2.375620,-0.641804,-0.627365,-0.385140,-0.374281
2,tcp,private,S0,anomaly,-0.807626,1.595477,1.600209,-0.372186,-0.373098,-1.388806,0.732059,-0.804947,-0.935081,-0.478183,1.603834,1.614454,-0.385140,-0.374281
3,tcp,http,SF,normal,1.238197,-0.193018,-0.187141,-0.372186,-0.373098,0.772109,-1.540854,1.264742,1.069663,-0.380894,-0.574435,-0.604947,-0.385140,-0.342768
4,tcp,http,SF,normal,1.238197,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,0.732059,1.264742,1.069663,-0.478183,-0.641804,-0.627365,-0.385140,-0.374281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,tcp,exec,RSTO,anomaly,-0.807626,-0.640142,-0.633978,2.765176,2.729322,-1.343313,0.732059,-0.976667,-1.091006,-0.478183,-0.641804,-0.627365,2.884296,2.777041
25188,tcp,ftp_data,SF,anomaly,1.238197,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-1.833807,-0.687453,1.069663,2.764775,-0.641804,-0.627365,-0.385140,-0.374281
25189,tcp,private,REJ,anomaly,-0.807626,-0.640142,-0.633978,2.765176,2.729322,-1.343313,0.732059,-0.922440,-1.046456,-0.478183,-0.641804,-0.627365,2.884296,2.777041
25190,tcp,nnsp,S0,anomaly,-0.807626,1.595477,1.600209,-0.372186,-0.373098,-1.184088,0.732059,-0.859174,-0.979631,-0.478183,1.603834,1.614454,-0.385140,-0.374281


In [ ]:
X = data.drop(['class'], axis=1)
y = data['class']
X

,protocol_type,service,flag,logged_in_zscore,serror_rate_zscore,srv_serror_rate_zscore,rerror_rate_zscore,srv_rerror_rate_zscore,same_srv_rate_zscore,dst_host_count_zscore,dst_host_srv_count_zscore,dst_host_same_srv_rate_zscore,dst_host_same_src_port_rate_zscore,dst_host_serror_rate_zscore,dst_host_srv_serror_rate_zscore,dst_host_rerror_rate_zscore,dst_host_srv_rerror_rate_zscore
0,tcp,ftp_data,SF,-0.807626,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.328634,-0.813985,-0.779157,0.073120,-0.641804,-0.627365,-0.221668,-0.374281
1,udp,other,SF,-0.807626,-0.640142,-0.633978,-0.372186,-0.373098,-1.320567,0.732059,-1.030895,-1.157831,2.375620,-0.641804,-0.627365,-0.385140,-0.374281
2,tcp,private,S0,-0.807626,1.595477,1.600209,-0.372186,-0.373098,-1.388806,0.732059,-0.804947,-0.935081,-0.478183,1.603834,1.614454,-0.385140,-0.374281
3,tcp,http,SF,1.238197,-0.193018,-0.187141,-0.372186,-0.373098,0.772109,-1.540854,1.264742,1.069663,-0.380894,-0.574435,-0.604947,-0.385140,-0.342768
4,tcp,http,SF,1.238197,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,0.732059,1.264742,1.069663,-0.478183,-0.641804,-0.627365,-0.385140,-0.374281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,tcp,exec,RSTO,-0.807626,-0.640142,-0.633978,2.765176,2.729322,-1.343313,0.732059,-0.976667,-1.091006,-0.478183,-0.641804,-0.627365,2.884296,2.777041
25188,tcp,ftp_data,SF,1.238197,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-1.833807,-0.687453,1.069663,2.764775,-0.641804,-0.627365,-0.385140,-0.374281
25189,tcp,private,REJ,-0.807626,-0.640142,-0.633978,2.765176,2.729322,-1.343313,0.732059,-0.922440,-1.046456,-0.478183,-0.641804,-0.627365,2.884296,2.777041
25190,tcp,nnsp,S0,-0.807626,1.595477,1.600209,-0.372186,-0.373098,-1.184088,0.732059,-0.859174,-0.979631,-0.478183,1.603834,1.614454,-0.385140,-0.374281


In [ ]:
#Splitting dataset into training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0, shuffle=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#One hot Encoding
trf1 = ColumnTransformer([
    ('ohe_state',OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

In [ ]:
X_train_dummy = trf1.fit_transform(X_train)
X_test_dummy = trf1.transform(X_test)
X_train_dummy

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 1.        ,  0.        ,  0.        , ..., -0.62736475,
        -0.38513984, -0.37428075],
       [ 0.        ,  0.        ,  1.        , ..., -0.62736475,
        -0.38513984, -0.37428075],
       [ 0.        ,  1.        ,  0.        , ..., -0.62736475,
        -0.38513984, -0.37428075],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  1.61445359,
        -0.38513984, -0.37428075],
       [ 0.        ,  1.        ,  0.        , ..., -0.62736475,
        -0.38513984, -0.37428075],
       [ 0.        ,  1.        ,  0.        , ..., -0.62736475,
        -0.38513984, -0.37428075]])

In [ ]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(contamination=0.05, random_state=42)  # Adjust contamination as needed
clf.fit(X_train_dummy,y_train)
outliers = clf.predict(X_test_dummy)
outlier_indices = np.where(outliers == -1)[0]
outlier_indices

array([   5,   21,   29,   34,   43,   44,   75,   82,   84,   97,   99,
        103,  133,  155,  160,  187,  195,  197,  216,  219,  252,  283,
        298,  308,  317,  331,  353,  362,  365,  393,  441,  443,  533,
        536,  542,  553,  554,  564,  575,  589,  611,  637,  663,  746,
        755,  765,  766,  776,  794,  845,  856,  859,  894,  902,  976,
        988, 1020, 1021, 1030, 1032, 1037, 1074, 1134, 1151, 1177, 1180,
       1232, 1281, 1295, 1299, 1320, 1358, 1374, 1375, 1377, 1402, 1421,
       1428, 1431, 1450, 1496, 1531, 1557, 1600, 1605, 1658, 1663, 1671,
       1688, 1690, 1703, 1712, 1722, 1765, 1767, 1805, 1809, 1811, 1831,
       1838, 1847, 1852, 1854, 1929, 1935, 1954, 1956, 1988, 2003, 2005,
       2024, 2036, 2064, 2068, 2080, 2088, 2101, 2106, 2110, 2114, 2148,
       2150, 2160, 2162, 2185, 2225, 2247, 2277, 2303, 2311, 2389, 2392,
       2405, 2430, 2437, 2442, 2444, 2446, 2451, 2521, 2553, 2554, 2581,
       2599, 2617, 2620, 2635, 2645, 2655, 2658, 27

In [ ]:
df_cleaned = data.drop(outlier_indices)
df_cleaned

,protocol_type,service,flag,class,logged_in_zscore,serror_rate_zscore,srv_serror_rate_zscore,rerror_rate_zscore,srv_rerror_rate_zscore,same_srv_rate_zscore,dst_host_count_zscore,dst_host_srv_count_zscore,dst_host_same_srv_rate_zscore,dst_host_same_src_port_rate_zscore,dst_host_serror_rate_zscore,dst_host_srv_serror_rate_zscore,dst_host_rerror_rate_zscore,dst_host_srv_rerror_rate_zscore
0,tcp,ftp_data,SF,normal,-0.807626,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.328634,-0.813985,-0.779157,0.073120,-0.641804,-0.627365,-0.221668,-0.374281
1,udp,other,SF,normal,-0.807626,-0.640142,-0.633978,-0.372186,-0.373098,-1.320567,0.732059,-1.030895,-1.157831,2.375620,-0.641804,-0.627365,-0.385140,-0.374281
2,tcp,private,S0,anomaly,-0.807626,1.595477,1.600209,-0.372186,-0.373098,-1.388806,0.732059,-0.804947,-0.935081,-0.478183,1.603834,1.614454,-0.385140,-0.374281
3,tcp,http,SF,normal,1.238197,-0.193018,-0.187141,-0.372186,-0.373098,0.772109,-1.540854,1.264742,1.069663,-0.380894,-0.574435,-0.604947,-0.385140,-0.342768
4,tcp,http,SF,normal,1.238197,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,0.732059,1.264742,1.069663,-0.478183,-0.641804,-0.627365,-0.385140,-0.374281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,tcp,exec,RSTO,anomaly,-0.807626,-0.640142,-0.633978,2.765176,2.729322,-1.343313,0.732059,-0.976667,-1.091006,-0.478183,-0.641804,-0.627365,2.884296,2.777041
25188,tcp,ftp_data,SF,anomaly,1.238197,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-1.833807,-0.687453,1.069663,2.764775,-0.641804,-0.627365,-0.385140,-0.374281
25189,tcp,private,REJ,anomaly,-0.807626,-0.640142,-0.633978,2.765176,2.729322,-1.343313,0.732059,-0.922440,-1.046456,-0.478183,-0.641804,-0.627365,2.884296,2.777041
25190,tcp,nnsp,S0,anomaly,-0.807626,1.595477,1.600209,-0.372186,-0.373098,-1.184088,0.732059,-0.859174,-0.979631,-0.478183,1.603834,1.614454,-0.385140,-0.374281


In [ ]:
x = df_cleaned.drop(['class'], axis=1)
y = df_cleaned['class']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0, shuffle=True)

In [ ]:
trf1 = ColumnTransformer([
    ('ohe_state',OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

In [ ]:
X_train = trf1.fit_transform(x_train)
X_test= trf1.transform(x_test)
X_train

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 0.        ,  1.        ,  0.        , ..., -0.62736475,
        -0.35244549, -0.37428075],
       [ 0.        ,  1.        ,  0.        , ..., -0.62736475,
        -0.38513984, -0.37428075],
       [ 0.        ,  1.        ,  0.        , ..., -0.62736475,
        -0.38513984, -0.37428075],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., -0.62736475,
        -0.38513984, -0.37428075],
       [ 0.        ,  1.        ,  0.        , ...,  1.61445359,
        -0.38513984, -0.37428075],
       [ 0.        ,  1.        ,  0.        , ..., -0.62736475,
        -0.38513984, -0.37428075]])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# create instances of all models
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB(),
    'Support Vector Machine': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Bagging': BaggingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Extra Trees': ExtraTreeClassifier(),
}


for name, md in models.items():
    md.fit(X_train_scaled,y_train)
    ypred = md.predict(X_test_scaled)

    print(f"{name}  with accuracy : {accuracy_score(y_test,ypred)}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression  with accuracy : 0.9705115346038115
Naive Bayes  with accuracy : 0.7021063189568706
Support Vector Machine  with accuracy : 0.9835506519558675
K-Nearest Neighbors  with accuracy : 0.9905717151454363
Decision Tree  with accuracy : 0.9909729187562688
Random Forest  with accuracy : 0.9929789368104313
Bagging  with accuracy : 0.9925777331995987
AdaBoost  with accuracy : 0.9662988966900702
Gradient Boosting  with accuracy : 0.9847542627883651
Extra Trees  with accuracy : 0.9889669007021064


In [ ]:
rfr = RandomForestClassifier()
rfr.fit(X_train_scaled,y_train)
rfr.predict(X_test_scaled)

array(['normal', 'normal', 'normal', ..., 'normal', 'anomaly', 'normal'],
      dtype=object)

In [ ]:
X_new = np.array([[0,'tcp','private',0,0,0.06,0,255,9,0.04,0.05,0,0,1,1,0,0]])
transformed_features = trf1.transform(X_new).reshape(1,-1)
X_new_scale = scaler.fit_transform(transformed_features)
y_pred2=rfr.predict(X_new_scale)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [ ]:
y_pred2

array(['normal'], dtype=object)